![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP/LargeFewShotClassifierModel.ipynb)

# 📜Large FewShot Classifier Model

This notebook will cover the different parameters and usages of `Large FewShot Classifier Model` .

**📖 Learning Objectives:**

1. Understand how to use `Large FewShot Classifier Model`.

2. Become comfortable using the different parameters of the annotator.

3. Build a pretraine pipeline using `Large FewShot Classifier Model` annotator.




## **📜 Background**


The new LargeFewShotClassifierModel annotator is designed to work effectively with minimal labeled data, offering flexibility and adaptability to new, unseen classes.



## **🎬 Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=False
nlp.install(refresh_install=True)

In [4]:
from johnsnowlabs import nlp, medical#, visual
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()
spark

👌 Detected license file /content/license_keys.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.4.0, 💊Spark-Healthcare==5.4.0, running on ⚡ PySpark==3.4.0


## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

##Pretrained Contextual Assertion Models





**<center>Large FewShot Classifier Models**


| Model Name              | Description |
|-------------------------|-------------|
|[large_fewshot_classifier_age_group](https://nlp.johnsnowlabs.com/2024/08/15/large_fewshot_classifier_age_group_en)      | This few-shot classification model identifies and classifies tweets reporting Adverse Drug Events (ADEs).
|[large_fewshot_classifier_ade](https://nlp.johnsnowlabs.com/2024/08/12/large_fewshot_classifier_ade_en)      | This few-shot classification model identifies and classifies the age group of a person mentioned in health documents.


**You can find all these models and more [NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=Chunk+Mapping&edition=Spark+NLP+for+Healthcare)**



## **🖨️ Input/Output Annotation Types**

- Input: `DOCUMENT`

- Output: `CATEGORY`

## Building Pipeline

In [5]:
#Pipeline for pretrained `large_fewshot_classifier_ade` model

documentAssembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

large_few_shot_classifier = medical.LargeFewShotClassifierModel()\
    .pretrained("large_fewshot_classifier_ade", "en", "clinical/models")\
    .setInputCols("document")\
    .setOutputCol("prediction")

pipeline = nlp.Pipeline().setStages([
    documentAssembler,
    large_few_shot_classifier
])

large_fewshot_classifier_ade download started this may take some time.
[OK!]


In [6]:
large_few_shot_classifier.getClasses()

['noADE', 'ADE']

In [7]:
data = spark.createDataFrame([
    ["The patient developed severe liver toxicity after taking the medication for three weeks"],
    ["He experienced no complications during the treatment and reported feeling much better."],
    ["She experienced a sudden drop in blood pressure after the administration of the new drug."],
    ["The doctor recommended a daily dosage of the vitamin supplement to improve her health."]
]).toDF("text")

result = pipeline.fit(data).transform(data)

result.selectExpr("text", "prediction.result[0] as prediction").show(truncate=False)

+-----------------------------------------------------------------------------------------+----------+
|text                                                                                     |prediction|
+-----------------------------------------------------------------------------------------+----------+
|The patient developed severe liver toxicity after taking the medication for three weeks  |ADE       |
|He experienced no complications during the treatment and reported feeling much better.   |noADE     |
|She experienced a sudden drop in blood pressure after the administration of the new drug.|ADE       |
|The doctor recommended a daily dosage of the vitamin supplement to improve her health.   |noADE     |
+-----------------------------------------------------------------------------------------+----------+

